from tkinter import *

app = Tk()
app.title('Tk')
app.geometry('')

app.iconbitmap(r'C:\Users\User\PycharmProjects\HelloWorld\my.ico')
app.mainloop()
pyinstaller --onefile -w -F --add-binary "test.png;." --add-binary "meNu.gif;." yt2mp3.py

In [6]:
import tkinter as tk
from ctypes import windll
from PIL import Image, ImageTk
from pytube import YouTube
import os
import re 

def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

class RootFrame(tk.Frame):
    def __init__(self, parent, title, logo:str="test.png", logo_sz:tuple=(50,50), font=None, **kwargs):
        self.parent = parent
        self.title = title
        self.logo = logo
        self.logo_sz = logo_sz # Must be int tuple
        super().__init__(self.parent, **kwargs)

        self.maximized = False
        self.hasstyle = False
        self.parent.windowSize = [self.parent.winfo_width(), 
                                  self.parent.winfo_height()]
        for key, val in kwargs.items():
            if key == 'highlightbackground':
                self.back_ground = val
            else:
                self.back_ground = "#2c2c2c"

        self.parent.withdraw()
        self.parent.update()
        dims = [int(x) for x in self.parent.geometry().split('+')[0].split('x')]
        dimension = (dims[0], dims[1])

        x = (self.parent.winfo_screenwidth()/2)-(dimension[0]/2)
        y = (self.parent.winfo_screenheight()/2)-250
        # self.parent.geometry(f'{dimension[0]}x{dimension[1]}+{int(x)}+{int(y)}')
        # self.parent.minsize(dimension[0], dimension[1])
        self.previousPosition = [int(x), int(y)]
        
        self.__ParentFrame__()
        self.__events__()
        self.loop_control()

    '''GUI Private Methods'''
    def __events__(self):
        self.title_bar.bind('<Double-1>', self.maximizeToggle)
        self.title_name.bind('<Double-1>', self.maximizeToggle)

        self.minimize_btn.bind('<Enter>', lambda x: self.minimize_btn.configure(bg='#777777'))
        self.minimize_btn.bind('<Leave>', lambda x: self.minimize_btn.configure(bg=self.back_ground))
        self.maximize_btn.bind('<Enter>', lambda x: self.maximize_btn.configure(bg='#777777'))
        self.maximize_btn.bind('<Leave>', lambda x: self.maximize_btn.configure(bg=self.back_ground))
        self.close_button.bind('<Enter>', lambda x: self.close_button.configure(bg='red'))
        self.close_button.bind('<Leave>', lambda x: self.close_button.configure(bg=self.back_ground))

    def __ParentFrame__(self):
        self.parent.overrideredirect(True)

        # title bar
        self.title_bar = tk.Frame(self.parent, bg=self.back_ground, bd=1,
                             highlightcolor=self.back_ground, 
                             highlightthickness=0)
        
        # Put stuff in titlebar
        # b) window logo
        img = Image.open(resource_path(self.logo))
        img = img.resize(self.logo_sz, Image.Resampling.LANCZOS)
        self.img = ImageTk.PhotoImage(img)
        self.panel = tk.Label(self.title_bar, image =self.img)
        self.panel.image=self.img
        
        # a) window title
        self.title_window = self.title
        self.title_name = tk.Label(self.title_bar, text=self.title_window, bg=self.back_ground, fg="white")
        
        #minimize btn
        self.minimize_btn = tk.Button(self.title_bar, text='🗕', bg=self.back_ground, padx=5, pady=2, 
                                 bd=0, font="bold", fg='white', width=2,
                                 activebackground="red",
                                 activeforeground="white", 
                                 highlightthickness=0, 
                                 command=self.minimize)

        #maximize btn
        self.maximize_btn = tk.Button(self.title_bar, text='🗖', bg=self.back_ground, padx=5, pady=2, 
                                 bd=0, font="bold", fg='white', width=2,
                                 activebackground="red",
                                 activeforeground="white", 
                                 highlightthickness=0, 
                                 command=self.maximizeToggle)

        #close btn
        self.close_button = tk.Button(self.title_bar, text='🗙', bg=self.back_ground, padx=5, pady=2, 
                                 bd=0, font="bold", fg='white', width=2,
                                 activebackground="red",
                                 activeforeground="white", 
                                 highlightthickness=0, 
                                 command= self.quit)
        # Set title bar at the top 
        self.title_bar.pack(side="top", fill="both", expand=True)
        
        # For each thing inside titlebar, pack it
        self.panel.pack(side='left', padx=5)
        self.title_name.pack(side='left', padx=5)
        self.close_button.pack(side='right')
        self.maximize_btn.pack(side=tk.RIGHT)
        self.minimize_btn.pack(side=tk.RIGHT)
        self.move_window_bindings(status=True)

    '''Functional Public Methods'''
    def get_pos(self, event):
        self.xwin = event.x
        self.ywin = event.y

    def loop_control(self):
        self.parent.update_idletasks()
        self.parent.withdraw()
        self.set_appwindow()

    def maximizeToggle(self, event=None):
        if self.maximized == False:
            self.winfo_update()
            #maximize current window
            self.maximize_btn.config(text="❐")
            hwnd = windll.user32.GetParent(self.parent.winfo_id())
            SWP_SHOWWINDOW = 0x40
            windll.user32.SetWindowPos(hwnd, 0, 0, 0, 
                int(self.parent.winfo_screenwidth()),
                int(self.parent.winfo_screenheight()-48),
                SWP_SHOWWINDOW)
            self.maximized = True
            self.move_window_bindings(status=False)
        else:
            #restore down window
            self.maximize_btn.config(text="🗖")
            hwnd = windll.user32.GetParent(self.parent.winfo_id())
            SWP_SHOWWINDOW = 0x40
            windll.user32.SetWindowPos(hwnd, 0, 
                self.previousPosition[0],
                self.previousPosition[1],
                int(self.parent.windowSize[0]),
                int(self.parent.windowSize[1]),
                SWP_SHOWWINDOW)
            self.maximized = False
            self.move_window_bindings(status=True)

    def minimize(self, hide=False):
       #reference: https://programtalk.com/python-examples/ctypes.windll.user32.ShowWindow/ 
        hwnd = windll.user32.GetParent(self.parent.winfo_id())
        windll.user32.ShowWindow(hwnd, 0 if hide else 6)

    def move_window(self, event):
        self.parent.geometry(f'+{event.x_root - self.xwin}+{event.y_root - self.ywin}')
        self.previousPosition = [self.parent.winfo_x(), self.parent.winfo_y()]

    def move_window_bindings(self, *args, status=True):
        if status == True:
            self.title_bar.bind("<B1-Motion>", self.move_window)
            self.title_bar.bind("<Button-1>", self.get_pos)
            self.title_name.bind("<B1-Motion>", self.move_window)
            self.title_name.bind("<Button-1>", self.get_pos)
        else:
            self.title_bar.unbind("<B1-Motion>")
            self.title_bar.unbind("<Button-1>")
            self.title_name.unbind("<B1-Motion>")
            self.title_name.unbind("<Button-1>")

    def quit(self):
        self.parent.destroy()

    def set_appwindow(self):
        GWL_EXSTYLE=-20
        WS_EX_APPWINDOW=0x00040000
        WS_EX_TOOLWINDOW=0x00000080
        if not self.hasstyle:
            hwnd = windll.user32.GetParent(self.parent.winfo_id())
            style = windll.user32.GetWindowLongW(hwnd, GWL_EXSTYLE)
            style = style & ~WS_EX_TOOLWINDOW
            style = style | WS_EX_APPWINDOW
            res = windll.user32.SetWindowLongW(hwnd, GWL_EXSTYLE, style)
            self.parent.withdraw()
            self.parent.after(10, lambda:self.parent.wm_deiconify())
            self.hasstyle=True

    def winfo_update(self):
        """Update geometry() information, return None"""
        self.parent.windowSize = [self.parent.winfo_width(),
                                  self.parent.winfo_height()]

class Application(tk.Frame):
    def __init__(self, master=None):
        self.master = master if master else tk.Tk()
        super().__init__(self.master)
        # Update master
        self.bg = "#2c2c2c"
        self.font = ('Helvetica', 8)
        self.rootFrame = RootFrame(self.master, title="Youtube to MP3", logo="test.png", logo_sz=(50,50),
                                bg="black", highlightthickness=1, highlightbackground=self.bg, font=self.font,)
        self.rootFrame.pack(side="top", fill="both", expand=True)
        self.windowFrame = tk.Frame(self.rootFrame, bg=self.bg,highlightcolor=self.bg, highlightthickness=0)
        self.windowFrame.pack(side="bottom", fill="both", expand=True)
        self.i = 0
        self.create_widgets()
        
    def create_widgets(self):
        # Static Label and input
        self.label = tk.Label(self.windowFrame, text="Youtube Link:", font=self.font,fg='white', bg=self.bg,highlightcolor=self.bg, highlightthickness=0)
        self.input = tk.Entry(self.windowFrame)
        self.input.insert(0, "Insert link here...")
        self.input.bind('<FocusIn>', self.clear_input)
        
        # Buttons
        self.mp3_button = tk.Button(self.windowFrame)
        self.mp3_button["text"] = "mp3"
        self.mp3_button["command"] = self.yt2mp3_mp3
        
        self.opus_button = tk.Button(self.windowFrame)
        self.opus_button["text"] = "opus"
        self.opus_button["command"] = self.yt2mp3_opus
        
        # Variable Labels
        self.output_label = tk.Label(self.windowFrame, text="", fg='white', bg=self.bg,highlightcolor=self.bg, highlightthickness=0)
        self.output_warn = tk.Label(self.windowFrame, text="", fg='red', bg=self.bg,highlightcolor=self.bg, highlightthickness=0)
        
        # Create gif
        self.create_gif()
        
        self.label.grid(row=0, column=0, sticky="news")
        self.input.grid(row=0, column=1, sticky="news")
        self.mp3_button.grid(row=0, column=2, sticky="news")
        self.opus_button.grid(row=0, column=3, sticky="news")
        self.output_label.grid(row=1, column=0, columnspan=4, sticky="news")
        self.output_warn.grid(row=2, column=0, columnspan=4, sticky="news")
        self.gif_label.grid(row=3, column=0, columnspan=4, sticky="news")
        
        # Start the gif
        self.windowFrame.after(0, self.update, 0)
    
    def create_gif(self):
        self.max_frames = 11
        self.frames = [
            tk.PhotoImage(file=resource_path("meNu.gif"), format="gif -index %i" % (i))
            for i in range(self.max_frames)
        ]
        self.gif_label = tk.Label(self.windowFrame, bg=self.bg,highlightcolor=self.bg, highlightthickness=0)

    def update(self, ind):
        frame = self.frames[ind]
        ind += 1
        if ind >= self.max_frames:  # With this condition it will play gif infinitely
            ind = 0
        self.gif_label.configure(image=frame)
        self.windowFrame.after(100, self.update, ind)
        
    def yt2mp3_mp3(self):
        self.youtube2mp3(codec_mp3=True)
    
    def yt2mp3_opus(self):
        self.youtube2mp3(codec_mp3=False)
        
    def youtube2mp3(self, codec_mp3:bool):
        url = self.input.get()
        try:
            yt = YouTube(url)
            title = yt.title

            if self.i == 0:
                happy_text = "Good Job honeyy" 
            elif self.i > 0 and self.i < 5:
                happy_text = f"Good Job honeyy x{self.i}!" 
            elif self.i >= 5 and self.i < 7:
                happy_text = f"Good Job HONEYYYY x99999 !!!!" 
            
            self.output_warn.config(fg='green')
            self.output_warn["text"] = happy_text
        except:
            self.i=0
            self.output_warn.config(fg='red')
            self.output_warn["text"] = "Baby Ahh, This is not a youtube link!"
            return
        title = re.sub(r'[^\w\s]', '', title)
        filetype = ".mp3" if codec_mp3 else ".opus"

        filtered = yt.streams.filter(only_audio=True)
        # Create empty dict
        desired_streams = {}

        for stream in filtered:
            curr_codec = 'mp4' if 'mp4' in stream.codecs[0] else stream.codecs[0]
            curr_abr = int(str(stream.abr).replace('kbps', ''))
            curr_tag = int(stream.itag)
            if curr_codec not in desired_streams.keys():
                desired_streams[curr_codec] = curr_abr, curr_tag
            else:
                if curr_abr > desired_streams[curr_codec][0]:
                    desired_streams[curr_codec] = curr_abr, curr_tag

        if len(desired_streams) > 1:
            self.output_label["text"] = 'Multiple codecs found, selecting the one with highest quality:'
            id_to_download = 0
            for key,value in desired_streams.items():
                if key == 'mp4':
                    continue
                else:
                    curr_bitrate, curr_id = value
                    mp4_bitrate , mp4_id = desired_streams['mp4']
                    id_to_download = curr_id if curr_bitrate < mp4_bitrate*2 else mp4_id
        else:
            vals = desired_streams.values()
            id_to_download = list(vals)[0][1]
            # print(f'Downloading stream with id: {id_to_download}')

        self.output_label["text"] = f"Begining Download . . ."
        stream = yt.streams.get_by_itag(id_to_download)
        stream.download(filename=title.strip(" | ")+filetype)
        self.output_label["text"] = f"Downloaded [{title}] "
    
    def clear_input(self, event):
        self.input.delete(0, 'end')

    def print_input(self):
        self.output_label["text"] = self.input.get()

if __name__ == '__main__':
    app = Application()
    app.mainloop()